In [1]:
from labcams import parse_cam_log, unpackbits,pjoin  # pip install labcams if it doesn't work
from glob import glob
import numpy as np
import h5py as h5
import pandas as pd
from scipy.interpolate import interp1d
import os

def export_for_matlab(sessionfolder, trialdata = None,camlog = None,camtime = None,eyedata = None):
    if trialdata is None:
        trialdata,camlog,camcomm,camtime,eyedata = load_data_from_droplets(sessionfolder)

    rewarded_side = np.array(trialdata.rewarded_side == 'left',dtype = int)
    rewarded_side[rewarded_side==0] = 2 # right side is 2 (left is 1)

    trial_start = np.array(trialdata.task_start_time)
    stim_onset = get_times_for_state_onset(trialdata,'stim') + np.array(trialdata.task_start_time)
    stim_duration = np.array(trialdata.stim_duration)
    response = np.array(trialdata.response)
    response[response==-1] = 2
    reward_time = get_times_for_state_onset(trialdata,'post_reward') + np.array(trialdata.task_start_time)

    rewarded = np.array(trialdata.rewarded)
    response_time = np.array(trialdata.response_time)
    timeout_time = get_times_for_state_onset(trialdata,'timeout') + np.array(trialdata.task_start_time)
    stim_intensity = np.array(trialdata.stim_intensity)
    stim_rates = np.array(trialdata.stim_rates)
    
    #fname = pjoin(sessionfolder,'analysis',sessionfolder, (subjectname, '_', sessionfolder, '_session_for_matlab.h5'))
    fname = pjoin(sessionfolder,'analysis',sessionfolder, '{}_session_for_matlab.h5'.format(sessionfolder))
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
    with h5.File(fname,'w') as fid:
        print('Saved {0}'.format(fname))
        fid.create_dataset('trialstart_times',data = trial_start)
        fid.create_dataset('frame_times',data = camtime)
        fid.create_dataset('pupil_diameter',data = eyedata['diameter'])
        fid.create_dataset('stim_onsets',data = stim_onset)
        fid.create_dataset('stim_duration',data = stim_duration)
        fid.create_dataset('reward_times',data = reward_time)
        fid.create_dataset('timeout_times',data = reward_time)
        fid.create_dataset('choice',data = response)
        fid.create_dataset('correct_side',data = rewarded_side)
        fid.create_dataset('rewarded', data = rewarded)
        fid.create_dataset('response_time', data = response_time)
        fid.create_dataset('stim_intensity', data = stim_intensity)

def get_times_for_state_onset(log, state = 'stim'):
    statetimes = []
    for i,l in log.iterrows():
        t = np.nan
        a = l['task_states']
        for s in a: # for each state
            if state in s:
                t = s[-1]
        statetimes.append(t)
    return np.array(statetimes)

def load_data_from_droplets(sessionfolder, dropletsfolder = 'DropletsTask', read_riglog = False):
    
    eyedata = {}
    camgpio = []

    with h5.File(glob(pjoin(sessionfolder,'*','*.mptracker.h5'))[0],'r') as pupildata:
        for k in pupildata.keys():
            eyedata[k] = pupildata[k][:]
    
    from labcams import parse_cam_log
    camlog,camcomm = parse_cam_log(glob(pjoin(sessionfolder,dropletsfolder,'*.camlog'))[0])

    camsyncmethod = 'trial_init'
    if 'var2' in camlog.columns:
        camgpio = unpackbits(np.array(camlog['var2'])) # the onsets
    if len(camgpio):
        if 2 in camgpio[0].keys():
            sync = camgpio[0][2]
            # check here if the trialstart was connected or it was the stim 
            if np.mean(camgpio[1][2]-camgpio[0][2]) > 15:
                camsyncmethod='stim'
                #print('Using the stim command to sync {0}.'.format(sessionfolder))
    if not 'sync' in locals():
        # then use the network command
        sync = []
        for c in camcomm:
            if 'trial_start' in c:
                sync.append(int(c.strip('# ').split(',')[0]))
        camsyncmethod = 'trial_init'
        print('Using the network command to sync {0}.'.format(sessionfolder))
        
    trialdata = pd.read_hdf(glob(pjoin(sessionfolder,dropletsfolder,'*triallog.h5'))[0])
    # get the onsets depending on the camsyncmethod
    trial_onsets = get_times_for_state_onset(trialdata,camsyncmethod)
    if camsyncmethod == 'trial_init':
        trial_onsets[:] = 0.
    trial_onsets += np.array(trialdata.task_start_time)  # add the start of each trial
    # get the frametimes syncronized with the trial onset.
    nframes = camlog.frame_id.iloc[-1]+1 # frame_id starts at zero
    if 'diameter' in eyedata.keys(): # use the size of diameter if available
        nframes = len(eyedata['diameter'])
    camtime = interp1d(sync[:len(trial_onsets)],trial_onsets, fill_value = 'extrapolate')(np.arange(nframes))
    
    return trialdata,camlog,camcomm,camtime, eyedata


In [2]:
datapath = r'C:\Users\Anne\data' # leave blank if working from the current working directory
subjectname = 'JC066'
sessionfolders = [os.path.dirname(os.path.dirname(r)) for r in glob(pjoin(
    datapath,subjectname,'*','*','*.mptracker.h5'))]
for sessionfolder in sessionfolders:
    trialdata,camlog,camcomm,camtime,eyedata = load_data_from_droplets(sessionfolders[0])
    export_for_matlab(sessionfolder)



Using the network command to sync C:\Users\Anne\data\JC066\20211006_151200.
Using the network command to sync C:\Users\Anne\data\JC066\20211006_151200.
Saved C:\Users\Anne\data\JC066\20211006_151200_session_for_matlab.h5
Using the network command to sync C:\Users\Anne\data\JC066\20211006_151200.
Using the network command to sync C:\Users\Anne\data\JC066\20211008_122338.
Saved C:\Users\Anne\data\JC066\20211008_122338_session_for_matlab.h5
Using the network command to sync C:\Users\Anne\data\JC066\20211006_151200.
Using the network command to sync C:\Users\Anne\data\JC066\20211012_165954.
Saved C:\Users\Anne\data\JC066\20211012_165954_session_for_matlab.h5
Using the network command to sync C:\Users\Anne\data\JC066\20211006_151200.
Using the network command to sync C:\Users\Anne\data\JC066\20211014_150318.
Saved C:\Users\Anne\data\JC066\20211014_150318_session_for_matlab.h5


In [ ]:
pd.read_hdf(r'C:\Users\Anne\data\JC047\20211001_114337_session_for_matlab.h5')

In [ ]:
trialdata